# Representacion en un modelo:
### Logistic Regresion

# data preprocessing

## 10 Threshold

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt


In [2]:
t10 = pd.read_csv('../data/Regression/t10.csv')
t20 = pd.read_csv('../data/Regression/t20.csv')
t50 = pd.read_csv('../data/Regression/t50.csv')
t100 = pd.read_csv('../data/Regression/t100.csv')

In [3]:
t10

,name,id,tournament_id,date,prizemoney,Knockout_stage,Knockout_stage_importance,player_1_id,player_1_name,elo_player_1,score_1,player_2_id,player_2_name,elo_player_2,score_2,Underdog
0,Arabia Invitational,1,15,2016-02-01,3000.0,Group Stage,1,29,TheViper,2282,3,18,St4rk,2051,2,False
1,Arabia Invitational,2,15,2016-02-01,3000.0,Group Stage,1,50,Zuppi,2039,3,28,Hera,1999,2,False
2,Arabia Invitational,3,15,2016-02-01,3000.0,Group Stage,1,35,vivi,1992,2,54,yinghua,2054,3,False
3,Arabia Invitational,4,15,2016-02-08,3000.0,Group Stage,1,29,TheViper,2276,3,50,Zuppi,2043,2,False
4,Arabia Invitational,5,15,2016-02-08,3000.0,Group Stage,1,18,St4rk,2057,3,35,vivi,1993,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13289,NAC CORP,13313,673,2024-02-02,NaN,Semi-Finals (Winner Bracket),1,1049,Nahuel05_,2034,2,1319,Parawhas,1971,0,False
13290,Swidden Cup,13447,685,2024-02-04,NaN,Semi-Finals (Winner Bracket),1,1635,Yakul,1900,1,1636,Kr1lle,1900,3,False
13291,Swidden Cup,13448,685,2024-02-04,NaN,Semi-Finals (Winner Bracket),1,65,Spaden,1951,2,731,mratin,1983,3,False
13292,India 1v1 Cup,13500,686,2024-03-09,NaN,Semi-Finals (Winner Bracket),1,603,The_Dragonstar,2147,2,1637,Ass_Kickerwa,1904,0,False


In [5]:
dfs = [t10, t20, t50, t100]

def preprocessData(dfs):
    for df in dfs:
        df.drop(['name','id','Knockout_stage','tournament_id','player_1_id','player_1_name','score_1','player_2_id','player_2_name','score_2',], axis=1,inplace=True)
        df['prizemoney'] = df['prizemoney'].replace(np.nan,0)
        df['s-tier'] = df['s-tier'].apply(lambda x: 1 if x else 0)
        df['hidden'] = df['hidden'].apply(lambda x: 1 if x else 0)
        df['Underdog'] = df['Underdog'].apply(lambda x: 1 if x else 0)

preprocessData(dfs)





In [6]:
t10

,prizemoney,Knockout_stage_importance,elo_player_1,elo_player_2,hidden,s-tier,Underdog
0,3000.0,1,2379,2038,0,0,0
1,3000.0,1,2089,2446,0,0,1
2,3000.0,1,2234,2023,0,0,1
3,3000.0,1,2379,2089,0,0,0
4,3000.0,1,2038,2234,0,0,1
...,...,...,...,...,...,...,...
10896,538.0,1,2006,1983,0,0,1
10897,1000.0,1,2160,2283,0,0,0
10898,1000.0,1,2369,2331,0,0,0
10899,338.0,1,2178,2111,0,0,1


In [13]:
t10.columns

Index(['prizemoney', 'Knockout_stage_importance', 'elo_player_1',
       'elo_player_2', 'hidden', 's-tier', 'Underdog'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.special import expit

In [8]:

def split(data):
    x_train, x_test, y_train, y_test = train_test_split(data.loc[:,data.columns!='Underdog'],data['Underdog'],test_size=0.2,random_state=1562)
    scaler = MinMaxScaler(feature_range=(0,0.1))
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    return x_train , x_test, y_train, y_test

def trainModel(x_train,y_train):
    model = LogisticRegression(max_iter=3000)
    model.fit(x_train,y_train)
    return model



In [9]:
data = []
for df in dfs:
    x_train, x_test, y_train, y_test  = split(df)
    model  = trainModel(x_train, y_train)
    #test model
    print(model.score(x_test,y_test))
    #print model equation in one line
    coeff = model.coef_
    inter = model.intercept_
    #save equation in array
    eq = f'y = {coeff[0][0]}*x1 + {coeff[0][1]}*x2 + {coeff[0][2]}*x3 + {coeff[0][3]}*x4 + {coeff[0][4]}*x5 + {coeff[0][5]}*x6 + {inter[0]}'
    data.append(eq) 
    

0.796423658872077
0.8248509857863365
0.8720770288858322
0.9307657038055938


In [12]:
for i, eq in enumerate(data):
    print(f"Ecuacion {i+1}: {eq}")
    

Ecuacion 1: y = 0.3048529023041543*x1 + 0.7406712944573912*x2 + -2.8223635439030286*x3 + 9.861325311144961*x4 + 0.37702367095271694*x5 + -2.1588752991635642*x6 + -1.6286974552632127
Ecuacion 2: y = 0.16980244516614232*x1 + 0.3055535504813268*x2 + -2.104395303972469*x3 + 9.328816038411585*x4 + 0.31686951361911164*x5 + -2.5275390327331095*x6 + -1.7696754830609378
Ecuacion 3: y = -0.10962246910067039*x1 + 0.14172311226271747*x2 + -0.2698038352269903*x3 + 8.075485879614597*x4 + 0.23930245627407176*x5 + -3.0204064136880078*x6 + -2.216242892278671
Ecuacion 4: y = -0.22033367156173855*x1 + -0.07470255139497131*x2 + 1.2382759483111518*x3 + 6.342724948169128*x4 + 0.07754355877103614*x5 + -2.13887484511026*x6 + -2.8562067876109807


In [56]:
t10.columns

Index(['prizemoney', 'Knockout_stage_importance', 'elo_player_1',
       'elo_player_2', 'hidden', 's-tier', 'Underdog'],
      dtype='object')

# Logistic Regression but with difference in features

# Normal Regression

## process data


In [58]:
t10 = pd.read_csv('../data/Regression/t10.csv')
t20 = pd.read_csv('../data/Regression/t20.csv')
t50 = pd.read_csv('../data/Regression/t50.csv')
t100 = pd.read_csv('../data/Regression/t100.csv')

In [57]:

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

In [59]:
dfs = [t10, t20, t50, t100]

def preprocessData(dfs):
    for df in dfs:
        df.drop(['name','id','Knockout_stage','tournament_id','player_1_id','player_1_name','player_2_id','player_2_name','Underdog'], axis=1,inplace=True)
        df['prizemoney'] = df['prizemoney'].replace(np.nan,0)
        df['s-tier'] = df['s-tier'].apply(lambda x: 1 if x else 0)
        df['hidden'] = df['hidden'].apply(lambda x: 1 if x else 0)

preprocessData(dfs)

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def split(data):
    X = data.drop(['score_1', 'score_2'], axis=1)
    y = data[['score_1', 'score_2']]
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1562)
    
    scaler = MinMaxScaler(feature_range=(0,0.1))
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    
    return x_train , x_test, y_train, y_test

,prizemoney,Knockout_stage_importance,elo_player_1,score_1,elo_player_2,score_2,hidden,s-tier
0,3000.0,1,2379,3,2038,2,0,0
1,3000.0,1,2089,3,2446,2,0,0
2,3000.0,1,2234,2,2023,3,0,0
3,3000.0,1,2379,3,2089,2,0,0
4,3000.0,1,2038,3,2234,1,0,0
...,...,...,...,...,...,...,...,...
10896,538.0,1,2006,0,1983,3,0,0
10897,1000.0,1,2160,1,2283,3,0,0
10898,1000.0,1,2369,3,2331,2,0,0
10899,338.0,1,2178,1,2111,3,0,0


In [70]:
data = []
for i, df in enumerate(dfs):
    x_train, x_test, y_train, y_test  = split(df)
    # Create a Linear Regression model
    model = LinearRegression()

    # Wrap the model with MultiOutputRegressor
    wrapper = MultiOutputRegressor(model)

    # Fit the model to the data
    wrapper.fit(x_train, y_train)
    
    print('\nScore Predictions- Player 1 and player 2: ', i)
    # Get the equations for each output
    for i in range(y_train.shape[1]):
        coef = wrapper.estimators_[i].coef_
        intercept = wrapper.estimators_[i].intercept_
        print(f'Output {i+1}: y = {coef} * x + {intercept}')
        





Score Predictions- Player 1 and player 2:  0
Output 1: y = [-9.57713786  3.73047004 19.88535293 -9.58157543  2.88652116  1.00462143] * x + 1.1940947611014234
Output 2: y = [ -5.83419053   3.78170923 -13.45903686  24.6616909    3.81018754
  -0.62986327] * x + 0.8218853179831802

Score Predictions- Player 1 and player 2:  1
Output 1: y = [-9.57713786  3.73047004 19.88535293 -9.58157543  2.88652116  1.00462143] * x + 1.1940947611014234
Output 2: y = [ -5.83419053   3.78170923 -13.45903686  24.6616909    3.81018754
  -0.62986327] * x + 0.8218853179831802

Score Predictions- Player 1 and player 2:  2
Output 1: y = [-9.57713786  3.73047004 19.88535293 -9.58157543  2.88652116  1.00462143] * x + 1.1940947611014234
Output 2: y = [ -5.83419053   3.78170923 -13.45903686  24.6616909    3.81018754
  -0.62986327] * x + 0.8218853179831802

Score Predictions- Player 1 and player 2:  3
Output 1: y = [-9.57713786  3.73047004 19.88535293 -9.58157543  2.88652116  1.00462143] * x + 1.1940947611014234
Outp